# Submit Predictions

This notebook shows how to prepare a submission of your model's predictions on the test data for the computation.

In [1]:
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))

from pathlib import Path
from natsort import natsorted
import numpy as np
from safetensors.torch import load_model
import torch 
import pandas as pd

from utils import hvatnet
from utils.creating_dataset import LEFT_TO_RIGHT_HAND


/home/lutetia/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load pre-trained model

This code loads the pre-trained baseline model - might be different for your model.

In [2]:
device = 'cuda:0'
dtype = torch.float32

# weights = r"C:\Users\feder\Documents\github\BCI_ALVI_challenge\tutorials\logs\test_2_run_fedya\step_3300_loss_0.2750.safetensors"
weights = "/media/lutetia/Extreme SSD/EMG_Yun/BCI_ALVI_challenge/tutorials/logs/optuna_run/step_31800_loss_0.1357.safetensors"

MODEL_TYPE = 'hvatnet'
model_config = hvatnet.Config(n_electrodes=8, n_channels_out=20,
                            n_res_blocks=3, n_blocks_per_layer=3,
                            n_filters=128, kernel_size=3,
                            strides=(2, 2, 2), dilation=2, 
                            small_strides = (2, 2))



# model_config = hvatnet.Config(n_electrodes=8, n_channels_out=20,
#                             n_res_blocks=5, n_blocks_per_layer=4,
#                             n_filters=25, kernel_size=4,
#                             strides=(2, 2, 2), dilation=1, 
#                             small_strides = (2, 2))

model = hvatnet.HVATNetv3(model_config)

# load_model(model, weights)
torch.load(weights, map_location=device)

model = model.to(device).to(dtype)

Number of parameters: 277850


### Save `submission.cvs` file

This code shows how the data was prepare and downsampled during inference. Make sure that your data is processed similarly!

In [3]:
# DATA_PATH = Path(r"F:\Dropbox (Personal)\BCII\BCI Challenges\2024 ALVI EMG Decoding\dataset_v2_blocks\dataset_v2_blocks")
DATA_PATH = Path("/media/lutetia/Extreme SSD/EMG_Yun/bci-initiative-alvi-hci-challenge/dataset_v2_blocks/dataset_v2_blocks")

test_data_name = 'fedya_tropin_standart_elbow_left'  # shoould match `test_dataset_list` used to train the model


data_folder = DATA_PATH / "amputant" / "left" / test_data_name / "preproc_angles" / "submit"
all_paths = natsorted(data_folder.glob('*.npz'))
print(f'Found {len(all_paths)} samples in {data_folder}')


Found 72 samples in /media/lutetia/Extreme SSD/EMG_Yun/bci-initiative-alvi-hci-challenge/dataset_v2_blocks/dataset_v2_blocks/amputant/left/fedya_tropin_standart_elbow_left/preproc_angles/submit


In [4]:

pred_list = []

# loop over each trial
for i, p in enumerate(all_paths):
    # get EMG data 
    sample = np.load(p)
    myo = sample['data_myo']
    myo = myo[:, LEFT_TO_RIGHT_HAND]

    # predictions will have to be downsampled
    gt_len = myo[::8].shape[0]

    # padding
    target_length = (myo.shape[0] + 255) // 256 * 256
    padded_myo = np.pad(myo, ((0, target_length - myo.shape[0]), (0, 0)), mode='constant', constant_values=0)

    # # print(f"Loaded data: {myo.shape} - padded to: {padded_myo.shape[0]}")
    # # Example: Trimming or reshaping `padded_myo`
    # desired_feature_size = 256
    # if padded_myo.shape[0] > desired_feature_size:
    #     # Trimming or reshaping the input to match the model's expected feature size
    #     padded_myo = padded_myo[:desired_feature_size, :]
    # # print(f"Trimmed to: {padded_myo.shape}")
    # # some prediction. might be slididng window.

    preds = model.inference(padded_myo)
    preds_downsampled = preds[:gt_len]
    print(f"Completed {i+1}/{len(all_paths)}. Loaded data: {myo.shape} - padded to: {padded_myo.shape} - predictions {preds.shape} - downsampled to: {preds_downsampled.shape}")
    pred_list.append(preds_downsampled)
    #print(pred_list[-1].shape)
#print(len(pred_list))
pred_cat = np.concatenate(pred_list, axis=0)
df = pd.DataFrame(pred_cat)
df.head()

/home/lutetia/anaconda3/lib/python3.9/site-packages/torch/nn/modules/conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv1d(input, weight, bias, self.stride,


Completed 1/72. Loaded data: (3721, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 2/72. Loaded data: (3725, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 3/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 4/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 5/72. Loaded data: (3729, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (467, 20)
Completed 6/72. Loaded data: (3726, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 7/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 8/72. Loaded data: (3728, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 9/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.309822,0.025852,-0.051900,-0.038487,-0.067631,0.420079,-0.153691,0.176609,-0.323555,0.139603,-0.061973,-0.027316,0.210803,0.265368,-0.272039,-0.105466,0.025260,0.205923,0.031897,0.014560
1,0.222320,-0.018878,-0.094138,-0.014115,-0.082424,0.368578,-0.180327,0.210975,-0.196762,0.161045,-0.122117,-0.027894,0.090236,0.248344,-0.334201,-0.137262,-0.001182,0.206787,0.002176,0.118896
2,0.295686,-0.074109,-0.157640,0.013762,-0.100583,0.283755,-0.158513,0.159621,-0.216080,0.127585,-0.125424,-0.042089,0.154597,0.248651,-0.190217,-0.076598,0.035310,0.162903,0.030259,0.119541
3,0.329360,-0.028401,-0.081093,0.053653,-0.084129,0.365069,-0.110618,0.241191,-0.289250,0.110976,-0.189444,-0.022342,0.066615,0.206617,-0.279613,-0.149786,0.004931,0.207335,0.030885,0.117385
4,0.299721,-0.086343,-0.126857,-0.043693,-0.034989,0.309056,-0.173559,0.203957,-0.174235,0.153253,-0.172447,0.010256,0.081476,0.245557,-0.373354,-0.127406,0.003140,0.163187,0.073052,0.165727


In addition to the predictions, your data should also include a sample id column.

In [5]:
df.insert(0, "sample_id", range(1, 1 + len(df)))

Finally, save to a CSV file. This is what you'll upload to Kaggle for the competition.

In [6]:
df.to_csv('submit_file_Wave_MSE_Optuna2.csv', index=False)

In [7]:
df.shape

(26829, 21)